First, I will import the libraries and packages needed for this assignment. 

In [1]:
#to scrape data from Wiki
from bs4 import BeautifulSoup

import numpy as np# library for data analsysis
import pandas as pd

#to handle json files
import json

#to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#to handle requests to webpages
import requests

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

#for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

#for clustering
from sklearn.cluster import KMeans

!pip -q install geocoder
# import time
import geocoder

# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('All good.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

Next, I will extract the table from the wikipedia page, and store it in my dataframe df.

In [33]:
#get wiki page
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wikipedia_link)

#extracts table from wiki page into df
df=pd.read_html(wikipedia_link, header=0)[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


Great! Now I have the df. But seems like it will need some cleaning up...

1. Remove cells where borough == NA
2. Where more than one neighborhood exists in a postal code area, combine those neighbourhoods into one row separated with a comma.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [34]:
#remove rows where borough == NA
df = df[df.Borough != "Not assigned"].reset_index()
df.drop(["index"], axis=1, inplace=True)

In [38]:
df.shape
df.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


For some reason, the original table on the wiki page already had the neighbourhoods with the same postal code grouped together, so there was no need for me to process that. 